In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
%run ./tools/test_net.py --gpu 0 --def models/pascal_voc/ResNet-50/rfcn_end2end/test_agnostic.prototxt --net /home/ubuntu/py-R-FCN/output/rfcn_end2end_ohem/voc_2007_trainval/resnet50_rfcn_ohem_iter_20000.caffemodel --imdb voc_2007_test --cfg experiments/cfgs/rfcn_end2end_ohem.yml

Called with args:
Namespace(caffemodel='/home/ubuntu/py-R-FCN/output/rfcn_end2end_ohem/voc_2007_trainval/resnet50_rfcn_ohem_iter_20000.caffemodel', cfg_file='experiments/cfgs/rfcn_end2end_ohem.yml', comp_mode=False, gpu_id=0, imdb_name='voc_2007_test', max_per_image=400, prototxt='models/pascal_voc/ResNet-50/rfcn_end2end/test_agnostic.prototxt', rpn_file=None, set_cfgs=None, vis=False, wait=True)
Using config:
{'DATA_DIR': '/home/ubuntu/py-R-FCN/data',
 'DEDUP_BOXES': 0.0625,
 'EPS': 1e-14,
 'EXP_DIR': 'rfcn_end2end_ohem',
 'GPU_ID': 0,
 'MATLAB': 'matlab',
 'MODELS_DIR': '/home/ubuntu/py-R-FCN/models/pascal_voc',
 'PIXEL_MEANS': array([[[ 102.9801,  115.9465,  122.7717]]]),
 'RNG_SEED': 3,
 'ROOT_DIR': '/home/ubuntu/py-R-FCN',
 'TEST': {'AGNOSTIC': True,
          'BBOX_REG': True,
          'HAS_RPN': True,
          'MAX_SIZE': 1000,
          'NMS': 0.3,
          'PROPOSAL_METHOD': 'selective_search',
          'RPN_MIN_SIZE': 16,
          'RPN_NMS_THRESH': 0.7,
          'RPN_PO

In [3]:
import cPickle  
with open('output/rfcn_end2end_ohem/voc_2007_test/resnet50_rfcn_ohem_iter_20000/test_preds.pkl','rb') as f:
    test_train_preds = cPickle.load(f)  

IOError: [Errno 2] No such file or directory: 'output/rfcn_end2end_ohem/voc_2007_test/resnet50_rfcn_ohem_iter_20000/test_preds.pkl'

In [ ]:
test_preds = test_train_preds[:1000]
train_preds = test_train_preds[1000:,:]

In [ ]:
FISH_CLASSES = ['NoF', 'ALB', 'BET', 'DOL', 'LAG','OTHER', 'SHARK', 'YFT']

with open("./data/VOCdevkit2007/VOC2007/ImageSets/Main/test.txt","r") as f:
    ims = f.readlines()
test_files = [im[:-1]+'.jpg' for im in ims[:1000]]
train_files = [[im[:-1] for im in ims[1000:]]]

with open("./data/VOCdevkit2007/VOC2007/ImageSets/Main/train_test.txt","r") as f:
    train_file_labels = f.readlines()

log_losses = []
for i in range(len(train_preds)):
    im = train_files[i]
    for im_label in train_file_labels:
        if im == im_label[:9]:
            label = im_label[-3:]
            index = FISH_CLASSES.index(label)
            log_losses.append(train_preds[i,index])
log_loss = sum(log_losses) / float(len(log_losses))
print('logloss of train is', logloss )

In [17]:
import datetime

submission = pd.DataFrame(test_preds, columns=FISH_CLASSES)
#submission.loc[:, 'image'] = pd.Series(test_files, index=submission.index)
submission.insert(0, 'image', test_files)

now = datetime.datetime.now()
sub_file = 'submission_RFCN_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
submission.to_csv(sub_file, index=False)